In [20]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve
from scipy.sparse import identity

In [21]:
Nx = 50  # Número de puntos en el dominio espacial
Nt = 2000  # Número de pasos de tiempo

H = 11.3
Tmax = 800
u = 1e-3

# Discretización espacial
dx = H / Nx
x = np.linspace(0, H, Nx + 1)  # Incluye extremos

# Discretización temporal
dt = Tmax / Nt

# Crear matrices dispersas para las derivadas espaciales
A = diags([1, -2, 1], [-1, 0, 1], shape=(Nx + 1, Nx + 1)) / dx**2
B = diags([1, -1], [0, 1], shape=(Nx + 1, Nx + 1)) / (2 * dx)

In [22]:
dx

0.226

In [52]:
# Inicializar matrices y vectores
T = np.zeros(Nx + 1)
Ts = np.zeros(Nx + 1) + Tmax

T[0] = Tmax  # Condición en x = 0

In [53]:
rhof = 1977
cpf = 700
eps = 0.22
rhos = rhof
cps = cpf
Lambda = 0.29

In [54]:
eps = 0.22
alpha = eps*rhof*cpf
beta = eps*Lambda
gamma = (1-eps)*rhos*cps
#hv = alpha*6*(1-eps)/d
hv = 66

In [55]:
# Matrices y vectores para las ecuaciones matriciales
I = identity(Nx + 1)
alpha_matrix = alpha / dt * I
beta_matrix = beta / 2 * (A + I)
u_alpha_matrix = u * alpha / 4 * B
hv_matrix = hv / 2 * I

In [59]:
# Crear matrices dispersas para las derivadas espaciales
A = diags([1, -2, 1], [-1, 0, 1], shape=(Nx + 1, Nx + 1)) / dx**2
B = diags([1, -1], [0, 1], shape=(Nx + 1, Nx + 1)) / (2 * dx)

# Matrices para almacenar soluciones en cada paso de tiempo (incluyendo extremos)
soluciones_T = np.zeros((Nt + 1, Nx + 1))
soluciones_Ts = np.zeros((Nt + 1, Nx + 1))

# Inicializar matriz de coeficientes para T y Ts
coef_matrix_T = alpha / dt * A + beta / 2 * (A + np.identity(Nx + 1)) - u * alpha / 4 * B
coef_matrix_Ts = np.identity(Nx + 1) + (gamma * hv * dt) * np.identity(Nx + 1)

# Establecer condiciones iniciales y de contorno para T
Tmax_array = np.full(Nx + 1, Tmax)  # Vector con Tmax en todas las posiciones
coef_matrix_T[0, :] = 0  # Fila de ceros en la primera fila
coef_matrix_T[0, 0] = 1  # Un 1 en la diagonal principal para la primera fila
coef_matrix_T[Nx, :] = 0  # Fila de ceros en la última fila
coef_matrix_T[Nx, Nx] = 1  # Un 1 en la diagonal principal para la última fila

# Establecer condiciones iniciales y de contorno para Ts
Ts_initial = np.zeros(Nx + 1)  # Puedes establecer los valores iniciales de Ts aquí si es necesario


In [61]:
# Iteración en el tiempo
for i in range(Nt + 1):
    # Almacenar soluciones en matrices (incluyendo extremos)
    soluciones_T[i] = Tmax_array
    soluciones_Ts[i] = Ts_initial

    # Resolver el sistema para T en el paso de tiempo actual
    T = spsolve(coef_matrix_T, Tmax_array)

    # Calcular Ts en el paso de tiempo actual utilizando el método de Euler
    Ts = Ts_initial + gamma * hv * dt * (Tmax_array - Ts_initial)

    # Actualizar las condiciones iniciales de Ts para el próximo paso de tiempo
    Ts_initial = Ts

C:\Users\MAAL1A\AppData\Local\anaconda3\Lib\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:214: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',
C:\Users\MAAL1A\AppData\Local\Temp\ipykernel_17776\441279516.py:11: RuntimeWarning: overflow encountered in multiply
  Ts = Ts_initial + gamma * hv * dt * (Tmax_array - Ts_initial)
C:\Users\MAAL1A\AppData\Local\Temp\ipykernel_17776\441279516.py:11: RuntimeWarning: invalid value encountered in add
  Ts = Ts_initial + gamma * hv * dt * (Tmax_array - Ts_initial)


In [62]:
soluciones_T

array([[800., 800., 800., ..., 800., 800., 800.],
       [800., 800., 800., ..., 800., 800., 800.],
       [800., 800., 800., ..., 800., 800., 800.],
       ...,
       [800., 800., 800., ..., 800., 800., 800.],
       [800., 800., 800., ..., 800., 800., 800.],
       [800., 800., 800., ..., 800., 800., 800.]])